In [22]:
import numpy as np
import pandas as pd

from dataqualitypipeline import DQPipeline
from dataqualitypipeline import initialize_autoencoder, initialize_autoencoder_modified

In [23]:
df_data = pd.read_csv("./HOWTO/players_20.csv")
df_data.head(1)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,...,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,NaN,NaN,NaN,NaN,NaN,NaN,"Beat Offside Trap, Argues with Officials, Earl...",88,95,70,92,88,97,...,91,84,93,95,95,86,68,75,68,94,48,40,94,94,75,96,33,37,26,6,11,15,14,8,89+2,89+2,89+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,93+2,92+2,87+2,87+2,87+2,92+2,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2


# FIFA World Cup Player Ratings
---
https://www.kaggle.com/datasets/thedevastator/fifa-world-cup-anomaly-detection-in-player-ratin

### Unsupervised Anomaly Detection on (train) data

In [34]:
only_columns = df_data.columns.to_list()
# only_columns[:-26]
only_columns[:37]

['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'nationality',
 'club',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'player_positions',
 'preferred_foot',
 'international_reputation',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'team_position',
 'team_jersey_number',
 'loaned_from',
 'joined',
 'contract_valid_until',
 'nation_position',
 'nation_jersey_number',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic']

In [35]:
from pyod.models.iforest import IForest
from pyod.models.lof import LOF
# import torch
clf_if = IForest(n_jobs=-1)
clf_ae = initialize_autoencoder_modified(epochs=200)
clf_lof = LOF(n_jobs=-1)

dq_pipe = DQPipeline(
    nominal_columns=["player_tags","preferred_foot",
                     "work_rate","team_position","loaned_from"],

    exclude_columns=["player_url","body_type","short_name", "long_name", 
                     "team_jersey_number","joined","contract_valid_until",
                     "real_face","nation_position","player_positions","nationality","club"],

    time_column_names=["dob"],
    deactivate_pattern_recognition=True,
    remove_columns_with_no_variance=True,
)

X_output = dq_pipe.run_pipeline(
    X_train=df_data.iloc[:,0:37],
    # X_train=df_data.iloc[:,:-26],
    clf=clf_lof,
    dump_model=False,
)



Using cpu device
Batch size: 8192
15 cores will be used...
Only X_train input will be transformed...
Running only Transformation-Pipeline...
No Variance in follow Train Columns:  Index(['Preprocessing Pipeline__Datetime__timeseries__X__X__dob_HOUR',
       'Preprocessing Pipeline__Datetime__timeseries__X__X__dob_MINUTE',
       'Preprocessing Pipeline__Datetime__timeseries__X__X__dob_SECOND',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_sofifa_id',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_player_url',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_short_name',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_long_name',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_age',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_dob',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_height_cm',
       'NaNMarker Pipeline__nan_marker_columns__missingindicator_weight_kg'

In [36]:
# dq_pipe.X_train_transformed

In [37]:
pd.set_option('display.max_columns', 100)

In [38]:
X_output.head(20)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,AnomalyScore,MAD_Total,Tukey_Total
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,93,93,58500000,405000,"ST, LW",Right,5,4,5,High/Low,C. Ronaldo,Yes,96500000.0,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2018-07-10,2022.0,LS,7.0,90.0,93.0,82.0,89.0,35.0,78.0,1.000000,9,11
1612,251691,https://sofifa.com/player/251691/alan-zamorado...,A. Zamorado,Alan Zamorado,38,1980-12-17,182,78,Ecuador,Ecuador,76,76,0,0,LM,Left,2,3,3,Medium/Low,Lean,No,NaN,"#Distance Shooter, #FK Specialist",NaN,NaN,NaN,NaN,NaN,LM,17.0,57.0,78.0,81.0,77.0,36.0,59.0,0.950809,0,2
17126,240117,https://sofifa.com/player/240117/maren-haile-s...,M. Haile-Selassie,Maren Haile-Selassie,20,1999-03-13,177,68,Switzerland,Neuchâtel Xamax,55,71,180000,1000,"LM, CAM",Right,1,2,3,Medium/Medium,Lean,No,NaN,#Acrobat,SUB,19.0,FC Zürich,NaN,2020.0,NaN,NaN,79.0,32.0,49.0,60.0,35.0,50.0,0.929625,0,0
2292,212273,https://sofifa.com/player/212273/clinton-njie/...,C. Njie,Clinton Mua N'Jie,25,1993-08-15,175,68,Cameroon,Cameroon,74,77,0,0,"ST, LW",Right,1,2,3,Medium/Low,Lean,Yes,NaN,"#Speedster, #Acrobat",NaN,NaN,NaN,NaN,NaN,LW,7.0,93.0,71.0,64.0,76.0,32.0,65.0,0.916918,0,1
1690,216820,https://sofifa.com/player/216820/moses-simon/2...,M. Simon,Moses Daddy Simon,23,1995-07-12,168,69,Nigeria,FC Nantes,75,81,9000000,21000,"LM, CF, RM",Right,1,5,4,Medium/Medium,Normal,No,NaN,"#Speedster, #Acrobat",LM,27.0,Levante UD,NaN,2020.0,NaN,NaN,92.0,70.0,64.0,81.0,36.0,63.0,0.892382,3,5
530,187936,https://sofifa.com/player/187936/steven-nzonzi...,S. Nzonzi,Steven N'Kemboanza Mike Christopher Nzonzi,30,1988-12-15,196,75,France,Galatasaray SK,80,80,12000000,68000,"CDM, CM",Right,2,3,2,Medium/Medium,Lean,Yes,NaN,"#Aerial Threat, #Strength",SUB,4.0,Roma,NaN,2020.0,NaN,NaN,42.0,55.0,71.0,68.0,79.0,86.0,0.873806,3,4
7060,229654,https://sofifa.com/player/229654/gerard-valent...,Gerard Valentín,Gerard Valentín Sancho,25,1993-07-28,177,75,Spain,CD Lugo,68,71,975000,4000,"RB, RM, RWB",Right,1,3,3,High/Medium,Normal,No,NaN,"#Speedster, #Acrobat",RB,20.0,Deportivo de La Coruña,NaN,2020.0,NaN,NaN,92.0,36.0,61.0,73.0,61.0,66.0,0.873776,0,0
6176,207645,https://sofifa.com/player/207645/modou-barrow/...,M. Barrow,Modou Barrow,26,1992-10-13,176,63,Gambia,Denizlispor,69,70,1200000,12000,"LM, LW",Left,1,2,3,Medium/Low,Lean,Yes,NaN,"#Speedster, #Acrobat",LM,77.0,Reading,NaN,2020.0,NaN,NaN,93.0,62.0,59.0,72.0,37.0,49.0,0.858023,0,1
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,94,94,95500000,565000,"RW, CF, ST",Left,5,4,4,Medium/Low,Messi,Yes,195800000.0,"#Dribbler, #Distance Shooter, #Crosser, #FK Sp...",RW,10.0,NaN,2004-07-01,2021.0,NaN,NaN,87.0,92.0,92.0,96.0,39.0,66.0,0.851553,8,11
6742,230079,https://sofifa.com/player/230079/aldo-kalulu/2...,A. Kalulu,Aldo Kalulu Kyatengwa,23,1996-01-21,166,66,France,Swansea City,68,77,1400000,10000,"RM, ST",Right,1,3,3,High/Low,Normal,No,NaN,"#Speedster, #Acrobat",LM,18.0,FC Basel 1893,NaN,2020.0,NaN,NaN,91.0,63.0,57.0,74.0,32.0,51.0,0.846326,0,1
